# Examples

In [1]:
import xevent
import numpy as np
import pandas as pd
import xarray as xr

In [2]:
import warnings

warnings.filterwarnings("ignore")

xr.set_options(display_expand_data=False)

np.random.seed(42)



## Example 1: Heat waves in a time series

In [3]:
# sample DataArray with dimension "time"
times = pd.date_range("2000-01-01", periods=100, freq="D")
da = xr.DataArray(np.random.rand(100), dims=["time"], coords={"time": times})
da

<xarray.DataArray (time: 100)>
0.3745 0.9507 0.732 0.5987 0.156 0.156 ... 0.4938 0.5227 0.4275 0.02542 0.1079
Coordinates:
  * time     (time) datetime64[ns] 2000-01-01 2000-01-02 ... 2000-04-09

In [4]:
# sample events dictionary
events = [
    {"time": np.datetime64("2000-01-11")},
    {"time": np.datetime64("2000-02-22")},
]

relative_time = pd.timedelta_range("-5D", "5D")
xevent.composite(da, events, relative_coords=dict(time=relative_time))

<xarray.DataArray (event: 2, rel_time: 11)>
0.156 0.05808 0.8662 0.6011 0.7081 ... 0.8948 0.5979 0.9219 0.08849 0.196
Coordinates:
    time      (event, rel_time) datetime64[ns] 2000-01-06 ... 2000-02-27
  * event     (event) int64 0 1
  * rel_time  (rel_time) timedelta64[ns] -5 days -4 days ... 4 days 5 days

## Example 2: Radiosonde profiles relative to tropopause

In [5]:
# sample DataArray with N radiosonde profiles with M levels each
N = 100
M = 30
profiles = pd.date_range("2000-01-01", periods=N)
levels = np.arange(M)

da = xr.DataArray(
    np.random.rand(N, M),
    dims=["profile", "level"],
    coords={"profile": profiles, "level": levels},
)
da

<xarray.DataArray (profile: 100, level: 30)>
0.03143 0.6364 0.3144 0.5086 0.9076 ... 0.6725 0.977 0.1019 0.00832 0.4336
Coordinates:
  * profile  (profile) datetime64[ns] 2000-01-01 2000-01-02 ... 2000-04-09
  * level    (level) int64 0 1 2 3 4 5 6 7 8 9 ... 20 21 22 23 24 25 26 27 28 29

In [6]:
tropopauses = [
    {"profile": np.datetime64("2000-01-11"), "level": 10},
    {"profile": np.datetime64("2000-02-22"), "level": 20},
]

relative_levels = np.arange(-5, 6)
xevent.composite(da, events=tropopauses, relative_coords=dict(level=relative_levels))

<xarray.DataArray (event: 2, rel_level: 11)>
0.8955 0.3892 0.01084 0.9054 0.09129 0.3193 ... 0.9756 0.641 0.8225 0.1325 0.862
Coordinates:
    profile    (event) datetime64[ns] 2000-01-11 2000-02-22
    level      (event, rel_level) int64 5 6 7 8 9 10 11 ... 19 20 21 22 23 24 25
  * event      (event) int64 0 1
  * rel_level  (rel_level) int64 -5 -4 -3 -2 -1 0 1 2 3 4 5

## Example 3: Time advantage compared to looping over events

In [7]:
# sample DataArray with nlon longitudes and nlat latitudes and ntime times
nlon = 360
nlat = 181
ntime = 100
lon = np.linspace(-180, 179, nlon)
lat = np.linspace(-90, 90, nlat)
time = pd.date_range("2000-01-01", periods=ntime)
da = xr.DataArray(
    np.random.rand(ntime, nlat, nlon),
    dims=["time", "lat", "lon"],
    coords={"time": time, "lat": lat, "lon": lon},
)
da

<xarray.DataArray (time: 100, lat: 181, lon: 360)>
0.09263 0.7484 0.9145 0.434 0.2587 0.4344 ... 0.247 0.9644 0.4387 0.8104 0.704
Coordinates:
  * time     (time) datetime64[ns] 2000-01-01 2000-01-02 ... 2000-04-09
  * lat      (lat) float64 -90.0 -89.0 -88.0 -87.0 -86.0 ... 87.0 88.0 89.0 90.0
  * lon      (lon) float64 -180.0 -179.0 -178.0 -177.0 ... 177.0 178.0 179.0

In [8]:
da.nbytes / 1e6  # in MB

52.128

In [9]:
events = [
    {"time": np.random.choice(time), "lat": np.random.choice(lat), "lon": np.random.choice(lon)} for _ in range(10_000)
]


In [10]:
rel_lon = np.arange(-5, 6)
rel_lat = np.arange(-5, 6)
rel_time = pd.timedelta_range("-5D", "5D")

%time composite = xevent.composite(da, events, relative_coords=dict(lon=rel_lon, lat=rel_lat, time=rel_time))
composite

CPU times: user 352 ms, sys: 54.8 ms, total: 407 ms
Wall time: 408 ms


<xarray.DataArray (event: 10000, rel_time: 11, rel_lat: 11, rel_lon: 11)>
0.5204 0.2114 0.9221 0.2079 0.2641 ... 0.002464 0.2175 0.7792 0.6119 0.283
Coordinates:
    time      (event, rel_time) datetime64[ns] 2000-01-25 ... 2000-02-09
    lat       (event, rel_lat) float64 74.0 75.0 76.0 77.0 ... -83.0 -82.0 -81.0
    lon       (event, rel_lon) float64 148.0 149.0 150.0 ... 151.0 152.0 153.0
  * event     (event) int64 0 1 2 3 4 5 6 ... 9993 9994 9995 9996 9997 9998 9999
  * rel_lon   (rel_lon) int64 -5 -4 -3 -2 -1 0 1 2 3 4 5
  * rel_lat   (rel_lat) int64 -5 -4 -3 -2 -1 0 1 2 3 4 5
  * rel_time  (rel_time) timedelta64[ns] -5 days -4 days ... 4 days 5 days

In [11]:
def looping_composite():
    result = []
    for e in events:
        event_center = da.sel(lon=e["lon"], lat=e["lat"], time=e["time"])
        subset = da.sel(
            lon=slice(e["lon"] - 5, e["lon"] + 5),
            lat=slice(e["lat"] - 5, e["lat"] + 5),
            time=slice(e["time"] - pd.Timedelta("5D"), e["time"] + pd.Timedelta("5D")),
        )
        rel_coords = dict(
            rel_lon=subset.lon - event_center.lon,
            rel_lat=subset.lat - event_center.lat,
            rel_time=subset.time - event_center.time,
        )
        result.append(subset.assign_coords(**rel_coords).swap_dims(lon="rel_lon", lat="rel_lat", time="rel_time"))
    return  xr.concat(result, dim="event").assign_coords(event=np.arange(len(events)))


%time xarray_result = looping_composite()
xarray_result

CPU times: user 1min 5s, sys: 213 ms, total: 1min 5s
Wall time: 1min 5s


<xarray.DataArray (event: 10000, rel_time: 11, rel_lat: 11, rel_lon: 11)>
0.5204 0.2114 0.9221 0.2079 0.2641 ... 0.002464 0.2175 0.7792 0.6119 0.283
Coordinates:
  * rel_lon   (rel_lon) float64 -5.0 -4.0 -3.0 -2.0 -1.0 0.0 1.0 2.0 3.0 4.0 5.0
  * rel_lat   (rel_lat) float64 -5.0 -4.0 -3.0 -2.0 -1.0 0.0 1.0 2.0 3.0 4.0 5.0
  * rel_time  (rel_time) timedelta64[ns] -5 days -4 days ... 4 days 5 days
    time      (event, rel_time) datetime64[ns] 2000-01-25 ... 2000-02-09
    lat       (event, rel_lat) float64 74.0 75.0 76.0 77.0 ... -83.0 -82.0 -81.0
    lon       (event, rel_lon) float64 148.0 149.0 150.0 ... 151.0 152.0 153.0
  * event     (event) int64 0 1 2 3 4 5 6 ... 9993 9994 9995 9996 9997 9998 9999

In [12]:
xr.testing.assert_equal(xarray_result.drop_vars(["lon", "lat", "time"]), composite.drop_vars(["lon", "lat", "time"]))